In [1]:
!pip install spylls
!pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 13.6 MB/s eta 0:00:00


In [4]:
import re
from collections import Counter
import tensorflow as tf
import numpy as np
from spylls.hunspell import Dictionary
from spellchecker import SpellChecker

# COMMON_WORDS жагсаалт
COMMON_WORDS = {"юм", "нь", "ба", "болон", "байна", "гэх", "бол"}

# Dummy classifier: Replace with your TensorFlow model and tokenizer
def load_dummy_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=10000, output_dim=64),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(4, activation='softmax')  # Economic, Sports, Literature, Politics
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = load_dummy_model()

def classify_text(text):
    if "спорт" in text.lower():
        return 1, "Спорт"
    elif "уран зохиол" in text.lower():
        return 2, "Уран зохиол"
    elif "эдийн засаг" in text.lower():
        return 0, "Эдийн засаг"
    elif "улс төр" in text.lower():
        return 4, "Улс төр"
    else:
        return 3, "Бусад"

def clean_text(text):
    return re.sub(r"[^\u0400-\u04FF\s]", "", text)  # Кирилл үсэг болон зайг үлдээх

# PySpellChecker ашиглах
spell_checker = SpellChecker(language=None)
spell_checker.word_frequency.load_text_file('/content/mn_MN.dic')

def check_spelling(word):
    """Check spelling using PySpellChecker"""
    if word in spell_checker:
        return True, []
    else:
        suggestions = spell_checker.candidates(word)
        return False, suggestions

def extract_root_words(text):
    words = clean_text(text).split()
    root_words = []
    misspelled = {}

    for word in words:
        is_correct, suggestions = check_spelling(word)
        root_words.append(word)
        if not is_correct:
            misspelled[word] = suggestions

    return root_words, misspelled

def process_text(text):
    root_words, misspelled = extract_root_words(text)
    word_counts = Counter(root_words)
    filtered_words = {word: count for word, count in word_counts.items() if word not in COMMON_WORDS}
    top_words = sorted(filtered_words.items(), key=lambda x: x[1], reverse=True)[:10]
    classification_id, category = classify_text(text)

    return {
        "misspelled": misspelled,
        "unique_words": len(set(root_words)),
        "top_words": top_words,
        "classification": category
    }

# Input text
text = """
Заримдаа аав ээжээсээ илүү үр хүүхэд нь хариуцлагатай байх ёстой ч юм шиг..
Хувь заяа (review)
Хувьсал продакшны киног үзэхийн тулд унд хоол, нус нулимсны алчуураа сайн бэлдэх хэрэгтэй. Учир нь тэд хэзээ ч минутандаа харам байдаггүй. Бас чамайг уйлуулж, ухааруулах гэж л кино туурвидаг.
"Хувь Заяа" кино нь эхийн хайраар дутсан өсвөр насны Хувьзаяа охины ээдрээт амьдралын талаар өгүүлэх драм кино бөгөөд найруулагч Г.Бямбасүрэнгийн ээлжит бүтээл юм.
Тус киноны үйл явдал Говийн нэгэн жижиг суманд өрнөх бөгөөд аавтайгаа сэтгэл хангалуун амьдардаг Хувьзаяа охины гэрт нэгэн үл мэдэгдэх бүсгүй ирснээр ээдрээт асуудлууд ар араасаа хөврөнө.
Ээжгүй охиныхоо сэтгэлийг дүүрэн байлгах гэж бүхий л амьдралаа зориулсан аав (Эрдэнэзаан) ганцхан удаа ч гэсэн охиноо элэг бүтэн амьдруулах гэсэн хүсэл мөрөөдөл нь эцэстээ юунд хүргэхийг тус киноноос үзээрэй.
Жүжигчин Эрдэнэзаантай юу ярихав. "Гавьяат" цол тэмдгээ чимж яваа нэгэн шүү дээ. Энэ кинон дээрх охиныхоо төлөө дотроо уйлж, гаднаа инээсэн аавын дүр нь яг л өөрөө мэт үнэмшилтэй санагдав.
Элсэн цөлд болоод ч тэрүү "Хувь Заяа" киноны зураг авалт, өнцөг шийдлүүд нь тун ч шинэлэг. Өдрийн зураг авалт, шөнийн зураг авалт, цаашлаад элсэн шуурга, зэрэглээ гээд л энэ киноны зураг авалт тэр чигтээ masterpiece байв.
Мөн цөөхөн хэрнээ чанартай хэдхэн жүжигчин нь гол зуртал үнэн жүжиглэх аж. Кино эхлээд 5 минут ч болоогүй байтал манай эхнэр "Жүжигчин Урнаа шиг гое хөгшрөх юмсан" гэж хэлсэн юмдаг. Нээрээ л энэ хүн хөгшрөхгүй нь. Дарс удах тусам чанараа хадгалдаг гэдэг шиг гавьяат маань улам л цэмбийсээр. Мөн Хувьзаяа охины дүрд тоглосон энэ бяцхан жүжигчин хэн бэ? Энэ насан дээр ийм мундаг дүрээ бүтээдэг хэнийг ч харж байсангүй ээ.
Нөгөө саахалт айлын санаа нэг гэдэг үгийн утгыг уран яруугаар тайлан харуулах агуу жүжигчин Мэндбаяр гуайг дурдахгүй өнгөрч болохгүй байх. Түүний дүр хөдөө нутгийн эгэл бор амьдралтай ч эгэлгүй сэтгэлтэй Монголчуудыг нэвт шувт илтгэж байлаа.
Тус киноны зохиол, хэлэх гэсэн санаа нь хамгийн ихээр сэтгэлд хүрч байв. Бид ер нь яагаад эцэг эхээ бидний төлөө амьдарч үхэх ёстой гэж боддог юм бэ? Заримдаа тэд ч бас хэн нэгнээр хайрлагдаж, аз жаргалын хэлтэрхийнээс ч болов хүртэх эрхтэй гэдгийг хожим нь ухаарах ёстойм гэж үү?
Ойрд гоё гоё драм кино үзээд гоё байна өө. Би чинь бас уян зөөлөн хүн юмсанж ххэ.
Яахав зарим хүмүүсийн сэтгэгдлийг ажаад байхад Хувьсалын кинонууд хэтэрхий хар бараан гэлцэх нь олон. Бид үзмээр байгаа киноогоо сонгоод үзэх боломжтой цаг үед амьдардаг шүү гэж л хэлье даа.
"""

# Process the input text and store the result
result = process_text(text)

# Print the misspelled words with suggestions
print("Misspelled Words with Suggestions:")
for word, suggestions in result["misspelled"].items():
    # `None` утгыг хоосон жагсаалт болгон хөрвүүлнэ
    suggestions_list = suggestions if suggestions is not None else []
    print(f" - {word}: {', '.join(suggestions_list)}")

# Print the total number of unique words
print("\nTotal Unique Words:", result["unique_words"])

# Print the top 10 frequent words, excluding common words
print("\nTop 10 Frequent Words (Excluding Common Words):")
for word, count in result["top_words"]:
    print(f" - {word}: {count}")

# Print the text classification
print("\nText Category:", result["classification"])


Misspelled Words with Suggestions:
 - Заримдаа: заримдаг
 - ээжээсээ: эндээсээ
 - хариуцлагатай: хариуцлагатан
 - байх: байт, байл, бах, бай, байц, байр, гайх, байз, барх, балх
 - ёстой: ёлтой, ёнтой, ёстон
 - продакшны: продакшн
 - киног: кино, кинг
 - үзэхийн: зэхийр, зэхийд, үзэсхий, зэхийт, зэхийл, зэхий
 - нулимсны: нулимс, нулимзна, нулимст
 - алчуураа: алчуудай, алчуур, алчууртан, алгуурла
 - бэлдэх: бэлдэц
 - хэрэгтэй: эрэгтэй, хэрэгтэн
 - минутандаа: 
 - байдаггүй: 
 - чамайг: чамай
 - уйлуулж: уйлуул
 - ухааруулах: ухааруулчих, ухааруулалц, ухааруул, ухааруулга
 - гэж: гэр, гэд, хэж, гэ, гэт, гэх, гэв, ээж, гэг, гэм, гээ, гэс, сэж, гүж, гөж, гэл, гаж, гж, эж, гож
 - туурвидаг: туурвигдагд, туурвигд
 - эхийн: хийт, тийн, хэхий, хийс, хэвийн, хэдийн, эхүүн, эвин, өхийд, эрхий, хийд, зэхий, нэхийт, элийр, охин, зэхийд, эвийл, ийн, мэхийр, еэхий, гэхий, юрийн, эхн, хэхийл, эхэн, бэхий, этий, сэхий, ёхий, тэхий, хайн, хийц, хийр, мэхийд, мэхий, эвий, энгийн, мийн, нэхий, зэхийр, х